# 介紹

在上一課中，我們學習了第一個基於模型的特徵工程方法：聚類。在本課中，我們將學習下一課：主成分分析 (PCA)。就像聚類是基於鄰近度對數據集進行分區(partitioning)一樣，我們可以將PCA 視為數據變化的分區工具，可以幫助我們發現數據中的重要關係，也可以用來創建更多信息特徵。

（技術說明：PCA 通常應用於[標準化](https://www.kaggle.com/alexisbcook/scaling-and-normalization)數據。標準化數據“變異(variation)”意味著“相關(correlation)”。非標準化數據“變異”表示“協方差(covariance)”。本課程中的所有數據將在應用 PCA 之前進行標準化。）


# 主成分分析(Principal Component Analysis)

在 [*Abalone*](https://www.kaggle.com/rodolfomendes/abalone-dataset) 數據集中是從數千個塔斯馬尼亞鮑魚中進行的物理測量。（鮑魚是一種海洋生物，很像蛤蜊或牡蠣。 ) 我們現在只看幾個特徵：它們殼的“高度”和“直徑”。

您可以想像在這些數據中有“變異軸”，它們描述了鮑魚彼此不同的方式。從圖片上看，這些軸顯示為沿著數據的自然維度延伸的垂直線，每個原始特徵一個軸。

<figure style = "padding: 1em;">
<img src = "https://i.imgur.com/rr8NCDy.png" 寬度 = 300, alt = "">
<figcaption style = "textalign: center; font-style: italic"> <center>
</center> </figcaption>
</figure>

通常，我們可以為這些變異軸命名。我們可以稱較長軸為“尺寸”成分：小高度和小直徑（左下）與大高度和大直徑（右上）形成對比。較短的軸我們可以稱為“形狀”成分：小高度和大直徑（扁平形狀）與大高度和小直徑（圓形）形成對比。

請注意，與其用“高度”和“直徑”來描述鮑魚，我們還可以用“尺寸”和“形狀”來描述它們。事實上，這就是 PCA 的全部思想：我們不是用原始特徵描述數據，**而是用它的變異軸來描述它。變異軸成為新特徵。**

<figure style="padding: 1em;">
<img src="https://i.imgur.com/XQlRD1q.png" width=600, alt="">
<figcaption style="textalign: center; font-style: italic"><center>主成分通過特徵空間中數據集的旋轉成為新特徵。
</center></figcaption>
</figure>

新特徵 PCA 構造實際上只是原始特徵的線性組合（加權和）：

``
df["尺寸"] = 0.707 * X["高度"] + 0.707 * X["直徑"]
df["形狀"] = 0.707 * X["高度"] - 0.707 * X["直徑"]
``

這些新特徵稱為數據的**主成分(principal components)**。權重本身被稱為 **loadings**。主成分與原始數據集中的特徵一樣多：如果我們使用十個特徵而不是兩個特徵，我們最終會得到十個成分。

一個成分的載荷告訴我們它通過符號和大小表達了什麼變化：

|功能\成分|尺寸 (PC1) |形狀 (PC2) |
|---------------|------------|-------------|
|身高 | 0.707 | 0.707 |
|直徑| 0.707 | -0.707 |


這張權重(loadings)表告訴我們，在“尺寸”成分中，“高度”和“直徑”在同一方向上變化（相同符號），但在“形狀”成分中，它們在相反方向上變化（相反符號）。在每個成分中，載荷的大小都相同，因此特徵在兩者中的貢獻相同。

PCA 還告訴我們每個成分的*數量*變化。從圖中我們可以看到，“Size”成分的數據變化比“Shape”成分的數據變化更大。 PCA 通過每個成分的 ** 解釋方差百分比** 使其精確。

<figure style="padding: 1em;">
<img src="https://i.imgur.com/xWTvqDA.png" width=600, alt="">
<figcaption style="textalign: center; font-style: italic"><center> Size 佔 96% 左右，Shape 佔 Height 和 Diameter 差異的 4% 左右。
</center></figcaption>
</figure>

`Size` 成分捕獲了 `Height` 和 `Diameter` 之間的大部分變化。然而，重要的是要記住，**成分中的方差量並不一定對應於它作為預測器的好壞：這取決於我們要預測的內容。**

# 用於特徵工程的 PCA

有兩種方法可以使用 PCA 進行特徵工程。

第一種方法是將其用作描述技術。由於成分會告訴我們有關變異的信息，因此可以計算成分的 MI 分數並查看哪種變異最能預測我們的目標。這可以為我們提供有關要創建的各種特徵的想法——如果“尺寸”很重要，我們可能會創建一個特徵為“高度”和“直徑”的乘積，或者如果“形狀”很重要，我可能會創建一個特徵為“直徑”和“高度”的比例乘積”。我們甚至可以嘗試對一個或多個高分成分進行聚類。

第二種方法是使用成分本身作為特徵。由於成分直接公開數據的變分結構，因此它們通常比原始特徵提供更多信息。以下是一些用例：
- **降維(Dimensionality reduction)**：當我們的特徵高度冗餘(redundancy)時（特別是*多重共線*），PCA 會將冗餘劃分為一個或多個接近零的方差分量，然後我們可以刪除這些分量，因為它們將包含很少或不相關信息。
- **異常檢測(Anomaly detection)**：原始特徵中不明顯的異常變化通常會出現在**低方差成分**中。這些成分在異常或異常值檢測任務中可能具有很高的信息量。
- **降噪(Noise reduction)**：傳感器讀數的集合通常會共享一些常見的背景噪音。 PCA 有時可以將（信息豐富的）信號收集到較少數量的特徵中，同時不考慮噪聲，從而提高信噪比。
- **降低相關性(Decorrelation)**：**一些 ML 算法在處理高度相關的特徵時遇到困難。 PCA 將相關特徵轉換為不相關的成分**，這對於我們的算法來說可能更容易使用。

PCA 基本上可以讓我們直接訪問數據的相關結構。毫無疑問，我們會想出自己的應用程序！

<blockquote style="margin-right:auto; margin-left:auto; background-color: #000000; padding: 1em; margin:24px;">
<strong>PCA 最佳做法</strong><br>
應用 PCA 時需要記住以下幾點：
<ul>
<li> PCA 僅適用於數字特徵，例如連續數量或計數。
<li> PCA 對比例敏感。在應用 PCA 之前對數據進行**標準化**是一種很好的做法，除非我們知道自己有充分的理由不這樣做。
<li> 考慮刪除或限制異常值，因為它們會對結果產生不當影響。
</ul>
</blockquote>

# 示例 - 1985 年的汽車 #

在此示例中，我們將返回到 [*Automobile*](https://www.kaggle.com/toramky/automobile-dataset) 數據集並應用 PCA，將其用作發現特徵的描述性技術。我們將在練習中查看其他用例。

這個隱藏的程式格加載數據並定義函數“plot_variance”和“make_mi_scores”。

In [2]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns
from IPython.display import display
from sklearn.feature_selection import mutual_info_regression


plt.style.use("seaborn-whitegrid")
plt.rc("figure", autolayout=True)
plt.rc(
    "axes",
    labelweight="bold",
    labelsize="large",
    titleweight="bold",
    titlesize=14,
    titlepad=10,
)


def plot_variance(pca, width=8, dpi=100):
    # Create figure
    fig, axs = plt.subplots(1, 2)
    n = pca.n_components_
    grid = np.arange(1, n + 1)
    # Explained variance
    evr = pca.explained_variance_ratio_
    axs[0].bar(grid, evr)
    axs[0].set(
        xlabel="Component", title="% Explained Variance", ylim=(0.0, 1.0)
    )
    # Cumulative Variance
    cv = np.cumsum(evr)
    axs[1].plot(np.r_[0, grid], np.r_[0, cv], "o-")
    axs[1].set(
        xlabel="Component", title="% Cumulative Variance", ylim=(0.0, 1.0)
    )
    # Set up figure
    fig.set(figwidth=8, dpi=100)
    return axs

def make_mi_scores(X, y, discrete_features):
    mi_scores = mutual_info_regression(X, y, discrete_features=discrete_features)
    mi_scores = pd.Series(mi_scores, name="MI Scores", index=X.columns)
    mi_scores = mi_scores.sort_values(ascending=False)
    return mi_scores


df = pd.read_csv("./data_set/Automobile_data.csv")

We've selected four features that cover a range of properties. Each of these features also has a high MI score with the target, `price`. We'll standardize the data since these features aren't naturally on the same scale.

In [4]:
df

,symboling,normalized-losses,make,fuel-type,aspiration,num-of-doors,body-style,drive-wheels,engine-location,wheel-base,...,engine-size,fuel-system,bore,stroke,compression-ratio,horsepower,peak-rpm,city-mpg,highway-mpg,price
0,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,13495
1,3,?,alfa-romero,gas,std,two,convertible,rwd,front,88.6,...,130,mpfi,3.47,2.68,9.0,111,5000,21,27,16500
2,1,?,alfa-romero,gas,std,two,hatchback,rwd,front,94.5,...,152,mpfi,2.68,3.47,9.0,154,5000,19,26,16500
3,2,164,audi,gas,std,four,sedan,fwd,front,99.8,...,109,mpfi,3.19,3.4,10.0,102,5500,24,30,13950
4,2,164,audi,gas,std,four,sedan,4wd,front,99.4,...,136,mpfi,3.19,3.4,8.0,115,5500,18,22,17450
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
200,-1,95,volvo,gas,std,four,sedan,rwd,front,109.1,...,141,mpfi,3.78,3.15,9.5,114,5400,23,28,16845
201,-1,95,volvo,gas,turbo,four,sedan,rwd,front,109.1,...,141,mpfi,3.78,3.15,8.7,160,5300,19,25,19045
202,-1,95,volvo,gas,std,four,sedan,rwd,front,109.1,...,173,mpfi,3.58,2.87,8.8,134,5500,18,23,21485
203,-1,95,volvo,diesel,turbo,four,sedan,rwd,front,109.1,...,145,idi,3.01,3.4,23.0,106,4800,26,27,22470


In [5]:
features = ["highway-mpg", "engine-size", "horsepower", "curb-weight"]

X = df.copy()
y = X.pop('price')
X = X.loc[:, features]

# Standardize
X_scaled = (X - X.mean(axis=0)) / X.std(axis=0)

/Users/huangshilun/opt/anaconda3/lib/python3.7/site-packages/ipykernel_launcher.py:8: FutureWarning: Dropping of nuisance columns in DataFrame reductions (with 'numeric_only=None') is deprecated; in a future version this will raise TypeError.  Select only valid columns before calling the reduction.
  


Now we can fit scikit-learn's `PCA` estimator and create the principal components. You can see here the first few rows of the transformed dataset.

In [ ]:
from sklearn.decomposition import PCA

# Create principal components
pca = PCA()
X_pca = pca.fit_transform(X_scaled)

# Convert to dataframe
component_names = [f"PC{i+1}" for i in range(X_pca.shape[1])]
X_pca = pd.DataFrame(X_pca, columns=component_names)

X_pca.head()

After fitting, the `PCA` instance contains the loadings in its `components_` attribute. (Terminology for PCA is inconsistent, unfortunately. We're following the convention that calls the transformed columns in `X_pca` the *components*, which otherwise don't have a name.) We'll wrap the loadings up in a dataframe.

In [ ]:
loadings = pd.DataFrame(
    pca.components_.T,  # transpose the matrix of loadings
    columns=component_names,  # so the columns are the principal components
    index=X.columns,  # and the rows are the original features
)
loadings

Recall that the signs and magnitudes of a component's loadings tell us what kind of variation it's captured. The first component (`PC1`) shows a contrast between large, powerful vehicles with poor gas milage, and smaller, more economical vehicles with good gas milage. We might call this the "Luxury/Economy" axis. The next figure shows that our four chosen features mostly vary along the Luxury/Economy axis.

In [ ]:
# Look at explained variance
plot_variance(pca);

Let's also look at the MI scores of the components. Not surprisingly, `PC1` is highly informative, though the remaining components, despite their small variance, still have a significant relationship with `price`. Examining those components could be worthwhile to find relationships not captured by the main Luxury/Economy axis.

In [ ]:
mi_scores = make_mi_scores(X_pca, y, discrete_features=False)
mi_scores

The third component shows a contrast between `horsepower` and `curb_weight` -- sports cars vs. wagons, it seems.

In [ ]:
# Show dataframe sorted by PC3
idx = X_pca["PC3"].sort_values(ascending=False).index
cols = ["make", "body_style", "horsepower", "curb_weight"]
df.loc[idx, cols]

To express this contrast, let's create a new ratio feature:

In [ ]:
df["sports_or_wagon"] = X.curb_weight / X.horsepower
sns.regplot(x="sports_or_wagon", y='price', data=df, order=2);

# Your Turn #

[**Improve your feature set**](https://www.kaggle.com/kernels/fork/14393921) by decomposing the variation in *Ames Housing* and use principal components to detect outliers.